# Validating Reachability

In [1]:
def isSolvable(input_grid):
  count_inversion = 0;
  grid_length = 9;
  if input_grid[-1] == 'B':
    grid_length = 8;
    for i in range(7):
      for j in range(i+1,8):
        if input_grid[i] == 'B' or input_grid[j] == 'B':
          continue;
        if (int(input_grid[i]) > int(input_grid[j])):
          count_inversion = count_inversion+1;
  return ((count_inversion%2) == 0);

# Genetic Algorithm

## Setting up Initial Population

In [2]:
import random
from queue import PriorityQueue
import time
import ast

In [3]:
def shuffle(test_list):
  res = random.sample(test_list, len(test_list))

  # Printing shuffled list
  # print (res)
  return res

## Fitting Function

In [5]:
def find_fit1(temp_list):
  sum = 0
  pointer = 1
  for i in temp_list:
    if i == "B":
      continue
    if(i != str(pointer)):
      sum+=1
    pointer+=1
  return [sum,temp_list]

In [11]:
def get_initial_pop(start_state):
  list_of_list = []
  mylist = start_state
  list_of_list = []
  list_of_list.append(mylist)

  while True:
    temp = shuffle(mylist)
    if(isSolvable(temp)):
      list_of_list.append(temp)
    if(len(list_of_list)>=10):
      break;

  # for each in list_of_list:
  #   print(each)
  # print(len(list_of_list))
  pq = PriorityQueue()

  for i in list_of_list:
    fit_pairs = find_fit1(i)
    # print("\n",fit_pairs)
    pq.put(fit_pairs)
  
  init_pop = []
  for i in pq.queue:
    init_pop.append(i)

  return init_pop

## Crossover and mutation to get new population

In [12]:
def do_crossover_mut(par1,par2):
  child = ['-1','-1','-1','-1','-1','-1','-1','-1','-1']
  temp_list = []
  for i in range(1,9):
    if(par1.index(str(i)) == i or par2.index(str(i)) == i):
      child[i-1] = str(i)
    else:
      temp_list.append(str(i))
  temp_list.append('B')  
  for i in temp_list:
    p1 = par1.index(i)
    p2 = par2.index(i)
    if(child[p1] == '-1'):
      child[p1] = i
    elif(child[p2] == '-1'):
      child[p2] = i
    else:
      child[child.index('-1')] = i

  return child

In [14]:
start_state = ['5','B','8','4','2','1','7','3','6']
target_state = [0,['1','2','3','4','5','6','7','8','B']]

In [15]:
def genetic_algo(start_state,target_state,runs):
  start_time = time.time()
  init_pop = get_initial_pop(start_state)
  curr_pop = init_pop

  while(runs>0):
    for each in curr_pop:
      if(each == target_state):
        print("\n\nTarget Found in, ",time.time()-start_time," And in ",11-runs,"Steps")
        return "Success"
    
    s = set()
    new_pq = PriorityQueue()
    for i in range(0,9):
      for j in range(1,10):
        child = do_crossover_mut(curr_pop[i][1],curr_pop[j][1])
        if(isSolvable(child)):
          s.add(str(child))      

    for i in s:
      child = ast.literal_eval(i)
      fit_pair = find_fit1(child)
      new_pq.put(fit_pair)


    new_gen = []
    ctr = 0
    while(ctr!=10):
      new_gen.append(new_pq.get())
      ctr+=1
    
    curr_pop = new_gen

    runs=runs-1

  return "Failure"  

In [16]:
def print_grid(current_grid):
  ctr = 1
  for i in current_grid:
    print(i,end = '   ')
    ctr = ctr+1
    if(ctr == 4 or ctr == 7):
      print('\n')

In [17]:
print("\n\nStart State\n")
print_grid(start_state)
print("\n\nTarget State\n")
print_grid(target_state[1])

status = genetic_algo(start_state,target_state,10)
print("\n\nAlgorithm Status:", status)



Start State

5   B   8   

4   2   1   

7   3   6   

Target State

1   2   3   

4   5   6   

7   8   B   

Target Found in,  0.0040395259857177734  And in  2 Steps


Algorithm Status: Success


# Simulated Annealing

In [18]:
from random import *
from math import *
from time import time

In [19]:
# Reaches in a single step
# start_state = [1,2,3,4,5,6,7,0,8]

# Does not necessarily reach
# start_state = [4,5,1,2,3,8,0,6,7]
start_state = [5,0,8,4,2,1,7,3,6]

target_state = [1,2,3,4,5,6,7,8,0]
heuristics = {0:"Misplaced Tiles",1:"Manhattan Distance"}


In [20]:

def get_row(pos):
        return (int(pos/3))

def get_col(pos):
        return int(pos%3)

def get_index(row, col):
        return row*3+col

    
class Node:

    goal_state = None
    start_state = None
    hn = None                              
        
    def __init__(self,state,parent,action):     
        
        self.state = state
        self.parent = parent
        self.action = action

   
    def all_pos_moves(self):
        blank_pos = self.state.index(0) 
        r = get_row(blank_pos)
        c = get_col(blank_pos)
        moves = ["U", "D", "L", "R"]
        if r == 0:
            moves.remove("U")
        if r == 2:
            moves.remove("D")
        if c == 0:
            moves.remove("L")
        if c == 2:
            moves.remove("R")
        return moves

    def do_move(self,move):
        new_state = self.state.copy()
        blank_pos = new_state.index(0)
        r = get_row(blank_pos)
        c = get_col(blank_pos)
        if move == "U":
            move_index = get_index(r-1, c)
        if move == "D":
            move_index = get_index(r+1, c)
        if move == "R":
            move_index = get_index(r, c+1)
        if move == "L":
            move_index = get_index(r, c-1)
        new_state[blank_pos],new_state[move_index] = new_state[move_index],new_state[blank_pos]
        return Node(new_state,self,move)

    def cal_e(self):
        
        md = 0
        mt = 0
        
        for i in range(9):
            
            if self.state[i] == 0:
                continue
            else:
                md += abs(self.state[i] - self.goal_state[i])
            
            if(self.state[i] != self.goal_state[i]): mt += 1
                
        
        if self.hn == 1:
            return md
        elif self.hn == 2:
            return mt
        else:
            return (md*mt)

        
    def Is_solved(self):
        if(self.state == self.goal_state):
            return True
        else:
            return False
    
    def trace_path(self):                    
        
        solution_list = []
        solution_list.append(self.action)            
        path = self
        while path.parent != None:              
            path = path.parent         
            solution_list.append(path.action)
        solution_list = solution_list[:-1]
        solution_list.reverse()
        return solution_list
    


In [21]:
def is_grid_valid(input_grid):
  count_inversion = 0;
  grid_length = 9;
  if input_grid[-1] == 0:
    grid_length = 8;
    for i in range(7):
      for j in range(i+1,8):
        if input_grid[i] == 0 or input_grid[j] == 0:
          continue;
        if (int(input_grid[i]) > int(input_grid[j])):
          count_inversion = count_inversion+1;
  return ((count_inversion%2) == 0);  

In [22]:
def print_grid(current_grid):
  ctr = 1
  for i in current_grid:
    print(i,end = '   ')
    ctr = ctr+1
    if(ctr == 4 or ctr == 7):
      print('\n')

In [23]:
def sim_ann(initial_state,goal_state, temp, hn):

    Node.start_state = initial_state
    Node.goal_state = goal_state
    Node.hn = hn
    temperature = temp
    state = Node(initial_state,None,None)   
    
    oldE = state.cal_e()                     
    n_explored = 0
    flag =False                              

    while temperature > 0:
        moves = state.all_pos_moves()             
        accepted = False
        
        while not accepted:

            m = choice(moves)              
            new_state = state.do_move(m)   
            n_explored+=1                  
            newE = new_state.cal_e()      
            
            deltaE = newE - oldE           
            if deltaE <= 0:
                accepted = True
                state = new_state          
            else:
                boltz = exp(-float(deltaE)/temperature)
                r = random()                          
                if r <= boltz:                         
                    accepted = True
                    state = new_state
        
        oldE = newE                         
        temperature = cool(temperature)    
        
        if state.Is_solved():                  
            flag = True
            break;
    
    optimal_path = state.trace_path()           
    return(optimal_path,n_explored,flag)

In [24]:
h_1 = 'Misplaced Tiles'
h_2 = 'Manhattan Distance'
h_3 = "h3 = h1 * h2"
heuristics[h_3] = 3

In [25]:
temp = 100

cool_f = 'T = T-1'
def cool(T):
    return (T-1)

In [ ]:

print("Start State")
print_grid(start_state)
print("\nGoal State")
print_grid(target_state)


if(is_grid_valid(start_state)):
  print("\nGrid is Solvable :)")
else:
  print("\nGrid is Unsovlable :(")

for hi in heuristics:
    print("\nHeuristics chosen:",heuristics[hi],"\nTemperature chosen:",temp,"\nCooling function:",cool_f,"\n")
    
    start_epoch = time()
    output = sim_ann(start_state, target_state, temp, heuristics[hi]) 
    exe_time = time() - start_epoch

    optimal_path = output[0]
    n_state = output[1]
    flag = output[2]

    if flag:
        print("Puzzle solved successfully\n")
        print('Optimal path:',optimal_path)

    else:
        print("Puzzle cant be solved\n")

    print('Total no. of states explored: ',n_state)
    print('\nTime taken: ', exe_time)
    print("\n\n")

Start State
5   0   8   

4   2   1   

7   3   6   
Goal State
1   2   3   

4   5   6   

7   8   0   
Grid is Solvable :)

Heuristics chosen: Misplaced Tiles 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle cant be solved

Total no. of states explored:  146

Time taken:  0.0016674995422363281




Heuristics chosen: Manhattan Distance 
Temperature chosen: 100 
Cooling function: T = T-1 

